In [1]:
#!/usr/bin/env python
'''
Created on 09/12/2014

@author: manuel.dominguezbecerra@nhs.net

'''


# Import libraries 
import argparse
import pandas as pd                      # pip install dash
import os
import subprocess
import numpy as np

import dash                              # pip install dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input

from dash_extensions import Lottie       # pip install dash-extensions
import dash_bootstrap_components as dbc  # pip install dash-bootstrap-components
import plotly.express as px              # pip install plotly
from datetime import date
import calendar
from wordcloud import WordCloud          # pip install wordcloud
import sqlite3                           # pip install sqlite3
import plotly.graph_objects as go
import dash_table
from dash.dependencies import Input, Output, State


# Take the samples path/name_file

#parser = argparse.ArgumentParser()
#parser.add_argument('--vcf1', type=str, required=True)
#parser.add_argument('--vcf2', type=int, required=True)
#args = parser.parse_args()

#Name_file1 = args.vcf1 # Remove comments when finished
#Name_file2 = args.vcf2

## Delete this when finished
#Name_file1 = "/Users/monkiky/Desktop/Do-these-samples-belong-to-the-same-patient/Samples/W2008872_S8-J4PJL_copy.vcf"
Name_file1 = "/Users/monkiky/Desktop/VCF-matcher/Samples/W2008872_S8-J4PJL.vcf"
#Name_file1 = "/Users/monkiky/Desktop/Do-these-samples-belong-to-the-same-patient/Samples/W2103014_S10-JGFJG.vcf"
Name_file2 = "/Users/monkiky/Desktop/VCF-matcher/Samples/Myeloid_1.2/210513_M00321_0553_000000000-JK4CY_Filtered_Annotated.vcf"


### 1. Remove the header of input files

# File A
cmd = "sed '/^##/ d'  {0} >  FileA.txt".format(Name_file1)
os.system(cmd)

# File B 
cmd = "sed '/^##/ d'  {0} >  FileB.txt".format(Name_file2)
os.system(cmd)

# Files to pandas object
dataA = pd.read_csv("./FileA.txt", delimiter = "\t" )
dataB = pd.read_csv("./FileB.txt", delimiter = "\t" )

# Delete files created
cmd = os. getcwd()+"/FileA.txt"
os.remove(cmd)

cmd = os. getcwd()+"/FileB.txt"
os.remove(cmd)

# If the pd dataframes have more than 10 columns
# that means there are more than 1 sample in the vcf file

if len(dataA.columns) > 10:
     Name_sample1 = input('It seems that {} has more than one sample. Please introduce the name of the sample you wish to analyse:  '.format(os.path.basename(Name_file1)))
else:
    print('File', os.path.basename(Name_file1), 'contains one sample only.')
    Name_sample1 = dataA.columns.tolist()[9]

if len(dataB.columns) > 10:
     Name_sample2 = input('It seems that {} has more than one sample. Please introduce the name of the sample you wish to analyse:  '.format(os.path.basename(Name_file2)))
else:
    print('File', os.path.basename(Name_file2), 'contains one sample only.')
    Name_sample2 = dataB.columns.tolist()[9]

    ### 2. Filter the variants  FILTER = PASS and Variant Frecuency =>0.4

# VF is in the 9th column, we need to extract and put VF data in an independent column first
df2 = dataA[Name_sample1].str.split(':', expand=True)

df2 = df2.fillna(value=np.nan)
df2 = df2.dropna(axis=1, how='any')

df2.columns = dataA.FORMAT.iloc[0].split(':')
dataA = pd.concat([dataA, df2], axis=1)

#df2.columns[df2.columns.str.contains(pat = 'VF')] 



# Same for dataB
df3 = dataB[Name_sample2].str.split(':', expand=True)

df3 = df3.fillna(value=np.nan) # Sometimes empty columns are created giving an error as there is more columns than columns names
df3 = df3.dropna(axis=1, how='any') # This delete the empty columns

df3.columns = dataB.FORMAT.iloc[0].split(':')
dataB = pd.concat([dataB, df3], axis=1)

# Now, let´s filter the variant
# We dont want variants that doesnt pass the Filter status
# This field is mandatory, no condition is needed because it is applied to any type of VCF file.
dataA = dataA.query('FILTER == "PASS"')
dataB = dataB.query('FILTER == "PASS"')
                    
# Now, I apply Variant frecuency (VF) filter (non-mandatory field)

# If there is a VF Genotype fields in the file
# Take the variants that are > 0.4
if "VF" in dataA.columns:
    dataA["VF"] = dataA["VF"].astype(float)
    dataA = dataA.query('VF > 0.4')
    
if "VF" in dataB.columns:
    dataB["VF"] = dataB["VF"].astype(float)
    dataB = dataB.query('VF > 0.4')
    

### 3. Let's create the Ben's Code, this is CHROM+POS+REF+ALT

dataA['CHROMPOSREDALT']=dataA["#CHROM"].apply(str)+"."+dataA["POS"].apply(str)+dataA["REF"]+"."+dataA["ALT"]

dataB['CHROMPOSREDALT']=dataB["#CHROM"].apply(str)+"."+dataB["POS"].apply(str)+dataB["REF"]+"."+dataB["ALT"]



### 4. Take common Ben's Codes in the same column and put the GT of each (both  copies) in independent columns
df3 = dataA[['CHROMPOSREDALT', 'GT']].copy()
df4 = dataB[['CHROMPOSREDALT', 'GT']].copy()
frames = [df3,df4]
semi_final_df = pd.concat(frames)


final_df=(semi_final_df.assign(key=semi_final_df.groupby('CHROMPOSREDALT').cumcount())
      .pivot('CHROMPOSREDALT','key','GT')
      .rename(columns=lambda x:f"Sample{x+1}")
      .rename_axis(columns=None).reset_index())



### 5. See how many GT match

if "Sample2" not in final_df.columns:
    print("No common position found in both samples")
    print("No report generated")
else:    
    final_df["Matches"] = np.where(final_df["Sample1"] == final_df["Sample2"], True, False)
    final_df.columns = ['CHROM.POS.REF.ALT',os.path.basename(Name_file1),os.path.basename(Name_file2),"Matches" ]



    
        
        
    
### 6.1 Generate results in the terminal

# Variable to introduce in the report:
    r0 = os.path.basename(Name_file1) # Get the  file name of a path/file_name 
    r1 = Name_sample1
    r2 = os.path.basename(Name_file2)
    r3 = Name_sample2
    r4 = final_df['Matches'].value_counts().get(True, 0) # Count trues if any, retunr 0
    r5 = final_df['Matches'].value_counts().get(False, 0)
    r6 = len(final_df)
    r7 = r4/(r4+r5)
    r6 == r4+r5
    report = '''
 _____________________________  REPORT  ________________________________________ 

vcf 1: {0}  AND its sample name: {1}  
vcf 2:  {2}  AND its sample name: {3}

                                                  Homozigous: 
Number of positions with the same genotype: {4} 
                                                  Heterozigous:
                                                
                                                  Homozigous:
Number of positions with diferent genotype: {5} 
                                                  Heterozigous: 


Total positions compared: {6}
Percentage in common: {4}/{6}= {7}
 ____________________________ END REPORT  _______________________________________
'''
    print(report.format(r0,r1,r2,r3,r4,r5,r6,r7))

File W2008872_S8-J4PJL.vcf contains one sample only.
It seems that 210513_M00321_0553_000000000-JK4CY_Filtered_Annotated.vcf has more than one sample. Please introduce the name of the sample you wish to analyse:  373977987
No common position found in both samples


In [28]:
df3 = dataB[Name_sample2].str.split(':', expand=True)

remove_empty_columns = len(df3.columns)-len(dataB.FORMAT.iloc[0].split(':'))
df3 = df3.iloc[: , :-remove_empty_columns]

df3.columns = dataB.FORMAT.iloc[0].split(':')
dataB = pd.concat([dataB, df3], axis=1)


Index(['VF'], dtype='object')

In [18]:
len(dataB.FORMAT.iloc[0].split(':'))

5

In [23]:
remove_empty_columns = len(df3.columns)-len(dataB.FORMAT.iloc[0].split(':'))
df3.iloc[: , :-remove_empty_columns]

,0,1,2,3,4
0,0/0,"4,0",4,12,"0,12,99"
1,0/0,"84,0",84,99,"0,120,1800"
2,0/1,"84,79",163,99,"1872,0,1877"
3,0/0,"201,0",201,99,"0,120,1800"
4,0/0,"205,0",205,99,"0,120,1800"
...,...,...,...,...,...
1213,0/0,"85,0",85,99,"0,120,1800"
1214,0/0,"56,0",56,99,"0,120,1800"
1215,0/0,"53,0",53,99,"0,120,1800"
1216,0/0,"89,0",89,99,"0,120,1800"


### 

In [ ]:

# NOOOOOOOO BORRARRRR ######################################


### 6. Generate a report in a dashboard
# Bootstrap themes by Ann: https://hellodash.pythonanywhere.com/theme_explorer
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SIMPLEX])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src='./assets/wrgllogohighres.jpeg')
                ],color="light", outline=True),
                ],width=3),
                dbc.Col([
                    dbc.Card([
                        dbc.CardImg(src='./assets/salisbury-nhs-foundation-trustLeft.png')
                             ],color="light", outline=True)
                        ],width=2)
            ],justify="between"),
        dbc.Row(
            [dbc.Card([ 
                dbc.CardBody([
                    html.Div([
                    html.H1("Report: Do the samples belong to the same patient?"),
                    html.H3("This application takes common variants and compares the GT fields. A relative large number of common variants which GT fields match is indicative that both samples have a close relationship."),
                ], style={'textAlign': 'center'}) 
                     ], style={'textLeft':'center', })      
                      ], style={'height':'10vh'},color="light", outline=True),
        ]),
        dbc.Row([
            dbc.Card([ 
                dbc.CardBody([
                    html.Div([
                    html.H1(""),
                    html.H3(""),
                ], style={'textAlign': 'center'}) 
                     ], style={'textLeft':'center', })      
                      ], style={'height':'10vh'},color="light", outline=True),
        ]),
        dbc.Row([  
            dbc.Col([
            dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": True, "selectable": True, "hideable": True}
            if i == "iso_alpha3" or i == "year" or i == "id"
            else {"name": i, "id": i, "deletable": True, "selectable": True}
            for i in final_df.columns
        ],
        data=final_df.to_dict('records'),  # the contents of the table html.H3(os.path.basename(Name_file1)),
        filter_action="native",     # allow filtering of data by user ('native') or not ('none')
        sort_action="native",       # enables data to be sorted per-column by user or not ('none')
        sort_mode="single",         # sort across 'multi' or 'single' columns
        row_deletable=True,         # choose if user can delete a row (True) or not (False)
        selected_rows=[],           # indices of rows that user selects
        page_action="native",       # all data is passed to the table up-front or not ('none')
        page_current=0,             # page number that user is on
        style_cell={                # ensure adequate header width when text is shorter than cell's text
            'minWidth': 200, 'maxWidth': 95, 'width': 95
        },
        style_data_conditional=[
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(248, 248, 248)'
        }
    ],
        style_header={
        'backgroundColor': 'rgb(230, 230, 230)',
        'fontWeight': 'bold'
    })]),
    dbc.Col([
        dbc.Card([ 
            dbc.CardBody([
                html.Div([
                    html.H4("Number of common variants:",style={'color': 'black', 'fontSize': 15}),
                    html.H4(len(final_df)),
                    html.H4("Number of matches:", style={'color': 'black', 'fontSize': 15}),
                    html.H4(final_df.Matches.value_counts()[1]),
                    ], 
                    style={'textAlign': 'left'})
                    ])
                    ],
                    style={'height':'8vh'},color="light", outline=True),
                    ])
                    ])
                    ])



  


if __name__ == '__main__':
    app.run_server(debug=False)


